## Final Project

#### 필요한 패키지들을 불러온다:

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics, preprocessing
from sklearn.decomposition import PCA
from scipy.stats import itemfreq

#### 데이터를 읽어온다:

In [2]:
os.chdir(r"C:\Users\bit\Desktop\20180717_머신러닝 알고리즘과 응용\data")

In [3]:
#df = pd.read_csv('data_kbo2015.csv', header='infer',encoding='ISO-8859-1')
df = pd.read_csv('data_activity_training.csv', header='infer',encoding='latin1')
# df_test = pd.read_csv('data_activity_testing.csv', header='infer',encoding='latin1')

C:\Users\bit\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (11,14,19,22,25,70,73,86,87,89,90,94,97,100) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print(df.shape)
#print(df_test.shape)

(19622, 160)


In [5]:
df.columns

Index(['Unnamed: 0', 'user_name', 'raw_timestamp_part_1',
       'raw_timestamp_part_2', 'cvtd_timestamp', 'new_window', 'num_window',
       'roll_belt', 'pitch_belt', 'yaw_belt',
       ...
       'gyros_forearm_x', 'gyros_forearm_y', 'gyros_forearm_z',
       'accel_forearm_x', 'accel_forearm_y', 'accel_forearm_z',
       'magnet_forearm_x', 'magnet_forearm_y', 'magnet_forearm_z', 'classe'],
      dtype='object', length=160)

#### 정상값이 3% 미만인 컬럼은 제거:

In [6]:
isnotZero = np.array(pd.isnull(df).mean(axis=0) < 0.97)
df = df.loc[:, isnotZero]

#### NA의 비율을 본다. 이제는 0!

In [7]:
pd.isnull(df).mean(axis=0)

Unnamed: 0              0.0
user_name               0.0
raw_timestamp_part_1    0.0
raw_timestamp_part_2    0.0
cvtd_timestamp          0.0
new_window              0.0
num_window              0.0
roll_belt               0.0
pitch_belt              0.0
yaw_belt                0.0
total_accel_belt        0.0
gyros_belt_x            0.0
gyros_belt_y            0.0
gyros_belt_z            0.0
accel_belt_x            0.0
accel_belt_y            0.0
accel_belt_z            0.0
magnet_belt_x           0.0
magnet_belt_y           0.0
magnet_belt_z           0.0
roll_arm                0.0
pitch_arm               0.0
yaw_arm                 0.0
total_accel_arm         0.0
gyros_arm_x             0.0
gyros_arm_y             0.0
gyros_arm_z             0.0
accel_arm_x             0.0
accel_arm_y             0.0
accel_arm_z             0.0
magnet_arm_x            0.0
magnet_arm_y            0.0
magnet_arm_z            0.0
roll_dumbbell           0.0
pitch_dumbbell          0.0
yaw_dumbbell        

#### Time label 제거 & 불필요 컬럼 제거:

In [8]:
header =df.columns
isnotTime = []
for col in header:
    if "time" in col:
        isnotTime.append(False)
    else:
        isnotTime.append(True)
df = df.loc[:, isnotTime]
#df_test = df_test.loc[:, isnotTime]
df = df.drop(columns = ['Unnamed: 0','user_name','new_window'])
#df_test = df_test.drop(columns = ['Unnamed: 0','user_name','new_window','problem_id'])
print(df.shape)
df.head(5)

(19622, 54)


,num_window,roll_belt,pitch_belt,yaw_belt,total_accel_belt,gyros_belt_x,gyros_belt_y,gyros_belt_z,accel_belt_x,accel_belt_y,...,gyros_forearm_x,gyros_forearm_y,gyros_forearm_z,accel_forearm_x,accel_forearm_y,accel_forearm_z,magnet_forearm_x,magnet_forearm_y,magnet_forearm_z,classe
0,11,1.41,8.07,-94.4,3,0.00,0.00,-0.02,-21,4,...,0.03,0.00,-0.02,192,203,-215,-17,654.0,476.0,A
1,11,1.41,8.07,-94.4,3,0.02,0.00,-0.02,-22,4,...,0.02,0.00,-0.02,192,203,-216,-18,661.0,473.0,A
2,11,1.42,8.07,-94.4,3,0.00,0.00,-0.02,-20,5,...,0.03,-0.02,0.00,196,204,-213,-18,658.0,469.0,A
3,12,1.48,8.05,-94.4,3,0.02,0.00,-0.03,-22,3,...,0.02,-0.02,0.00,189,206,-214,-16,658.0,469.0,A
4,12,1.48,8.07,-94.4,3,0.02,0.02,-0.02,-21,2,...,0.02,0.00,-0.02,189,206,-214,-17,655.0,473.0,A


In [9]:
LE = preprocessing.LabelEncoder()
X = np.array(df.drop(columns='classe'))
Y = np.array(df.loc[:,'classe'])
Y = LE.fit_transform(Y)

#### KNN 적용:

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=3)
knn = KNeighborsClassifier(n_neighbors=5, weights ='distance')
knn.fit(X_train, Y_train);
Y_pred = knn.predict(X_test)
print( "Accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))

Accuracy : 0.945


#### KNN의 매개변수 최적화:

In [ ]:
k_grid = np.arange(1,51,1)
weights = ['uniform','distance']
parameters = {'n_neighbors':k_grid, 'weights':weights}

In [ ]:
gridCV = GridSearchCV(KNeighborsClassifier(), parameters, cv=10)
gridCV.fit(X_train, Y_train);
best_k = gridCV.best_params_['n_neighbors']
best_w = gridCV.best_params_['weights']

In [ ]:
print("Best k : " + str(best_k))
print("Best weight : " + best_w)

In [ ]:
knn_best = KNeighborsClassifier(n_neighbors=best_k, weights = best_w)
knn_best.fit(X_train, Y_train);
Y_pred = knn_best.predict(X_test)
print( "Best Accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))

#### KNN + PCA 적용:

In [13]:
pca = PCA(n_components = 40)
X_pca = pca.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X_pca,Y, test_size=0.3, random_state=3)
knn = KNeighborsClassifier(n_neighbors=5, weights ='distance')
knn.fit(X_train, Y_train);
Y_pred = knn.predict(X_test)
print( "Accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))

Accuracy : 0.945
